In [1]:
# Here we will make confidence intervals for the feedbacks/transports

# By: Ty Janoski
# updated: 06.23.20

In [2]:
# import statements
import xarray as xr
import scipy.stats as stats
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
%config InlineBackend.figure_format = "pdf"

In [3]:
models = ['ACCESS1-0','ACCESS1-3','CNRM-CM5','IPSL-CM5B-LR', 'GFDL-ESM2G',
         'MIROC-ESM', 'FGOALS-g2','bcc-csm1-1','BNU-ESM','CanESM2','CCSM4',
          'CSIRO-Mk3-6-0','FGOALS-s2','GFDL-CM3','inmcm4',
         'IPSL-CM5A-LR','MIROC5','MPI-ESM-LR','MPI-ESM-P','MRI-CGCM3','NorESM1-M']
arc_norm_list = []
glb_norm_list = []
arc_ml_list = []
glb_ml_list = []
for m in models:
    # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
    arc = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_arc_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    glb = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                         use_cftime=True,drop_variables=['ml_conv'])
    # to convert to temperature contributions, we normalize by the global Planck feedbacks
    # divided by the global change in surface skin temp
    norm = (glb.planck_LWAS / glb.dTs)
    
    arc_norm = -1 * arc / norm
    glb_norm = -1 * glb / norm
    
    # a bit of wonkiness: planck is actually the deviation from the global planck feedback
    arc_norm['planck_LWAS'] = -1 * ((arc.planck_LWAS / arc.dTs) - (glb.planck_LWAS / glb.dTs)) * arc.dTs / norm
    glb_norm['planck_LWAS'] = -1 * ((glb.planck_LWAS / glb.dTs) - (glb.planck_LWAS / glb.dTs)) * glb.dTs / norm
    
    # it would be useful to combine each model into one dataset with a "model" dimension
    # each model's calendar is different, so unfortunately we will override the time dimension
    arc_norm['time'] = np.arange(0,1800,1)
    glb_norm['time'] = np.arange(0,1800,1)
    
    # append to lists
    arc_norm_list.append(arc_norm)
    glb_norm_list.append(glb_norm)
    
    ot_models = ['ACCESS1-0','ACCESS1-3','GFDL-CM3','MIROC5',
             'MPI-ESM-LR','MPI-ESM-P','MIROC-ESM','BNU-ESM',
             'CCSM4','MRI-CGCM3','FGOALS-g2',
             'CNRM-CM5','NorESM1-M','CSIRO-Mk3-6-0']
    if(m in ot_models):
        print(m)
        # read in spatially averaged feedbacks, as computed in spatial_avg_feedbacks.ipynb
        arc_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_arc_feedbacks.nc',
                             use_cftime=True).ml_conv
        glb_ml = xr.open_dataset('/dx07/tylerj/CMIP5_output/CMIP5_feedbacks/'+m+'_glb_feedbacks.nc',
                             use_cftime=True).ml_conv
        arc_ml_norm = -1 * arc_ml/norm
        glb_ml_norm = -1 * glb_ml/norm
        arc_ml_norm['time'] = np.arange(0,1800,1)
        glb_ml_norm['time'] = np.arange(0,1800,1)
        arc_ml_list.append(arc_ml_norm)
        glb_ml_list.append(glb_ml_norm)
# combine along new model coordinate
arc_temp = xr.concat(arc_norm_list,dim='model')
arc_temp['model'] = models
glb_temp = xr.concat(glb_norm_list,dim='model')
glb_temp['model'] = models
arc_ml_temp = xr.concat(arc_ml_list,dim='model')
arc_ml_temp['model'] = ot_models
glb_ml_temp = xr.concat(glb_ml_list,dim='model')
glb_ml_temp['model'] = ot_models


ACCESS1-0
ACCESS1-3
CNRM-CM5
MIROC-ESM
FGOALS-g2
BNU-ESM
CCSM4
CSIRO-Mk3-6-0
GFDL-CM3
MIROC5
MPI-ESM-LR
MPI-ESM-P
MRI-CGCM3
NorESM1-M


In [4]:
# take difference between arctic and global temperature contributions

diff = arc_temp - glb_temp
diff_ml = arc_ml_temp - glb_ml_temp

In [5]:
mean3mon = []
error3mon = []
mean1yr = []
error1yr = []
mean30yr = []
error30yr = []

for var in ['planck_LWAS','lapse_LWAS','albedo_SWAS','wv_NETAS','cloud_NET','atmos_conv','ocean_conv']:
    if(var!='ocean_conv'):
        mean = diff[var].isel(time=slice(0,3)).mean(dim=['time','model'])
        std = diff[var].isel(time=slice(0,3)).mean(dim='time').std(dim='model')/np.sqrt(len(diff.model))
        interval = stats.t.interval(0.95,20,loc=mean,scale=std)
        mean3mon.append(float(mean))
        error3mon.append((interval[1] - interval[0])/2)
        
        mean = diff[var].isel(time=slice(0,12)).mean(dim=['time','model'])
        std = diff[var].isel(time=slice(0,12)).mean(dim='time').std(dim='model')/np.sqrt(len(diff.model))
        interval = stats.t.interval(0.95,20,loc=mean,scale=std)
        mean1yr.append(float(mean))
        error1yr.append((interval[1] - interval[0])/2)
        
        mean = diff[var].isel(time=slice(1200,1800)).mean(dim=['time','model'])
        std = diff[var].isel(time=slice(1200,1800)).mean(dim='time').std(dim='model')/np.sqrt(len(diff.model))
        interval = stats.t.interval(0.95,20,loc=mean,scale=std)
        mean30yr.append(float(mean))
        error30yr.append((interval[1] - interval[0])/2)
    else:
        mean = diff_ml.isel(time=slice(0,3)).mean(dim=['time','model'])
        std = diff_ml.isel(time=slice(0,3)).mean(dim='time').std(dim='model')/np.sqrt(len(diff_ml.model))
        interval = stats.t.interval(0.95,20,loc=mean,scale=std)
        mean3mon.append(float(mean))
        error3mon.append((interval[1] - interval[0])/2)
        
        mean = diff_ml.isel(time=slice(0,12)).mean(dim=['time','model'])
        std = diff_ml.isel(time=slice(0,12)).mean(dim='time').std(dim='model')/np.sqrt(len(diff_ml.model))
        interval = stats.t.interval(0.95,20,loc=mean,scale=std)
        mean1yr.append(float(mean))
        error1yr.append((interval[1] - interval[0])/2)
        
        mean = diff_ml.isel(time=slice(1200,1800)).mean(dim=['time','model'])
        std = diff_ml.isel(time=slice(1200,1800)).mean(dim='time').std(dim='model')/np.sqrt(len(diff_ml.model))
        interval = stats.t.interval(0.95,20,loc=mean,scale=std)
        mean30yr.append(float(mean))
        error30yr.append((interval[1] - interval[0])/2)

In [6]:
fig,axes = plt.subplots(nrows=3,figsize=(3.7,6))
xs = np.arange(0,7,1)
colors = ['darkorange','green','red','purple','turquoise','orchid','Navy']
axes[0].hlines(0,-1,7,color='black')
axes[0].errorbar(xs,mean3mon,yerr=error3mon,ecolor=colors,fmt='none')
axes[0].scatter(xs,mean3mon,c=colors)
axes[0].set_xlim([-0.5,6.5])
axes[0].set_xticks(xs)
axes[0].set_yticks(np.arange(-1.5,1.6,0.5))
axes[0].set_ylim([-1.5,1.5])
axes[0].xaxis.set_tick_params(labelsize=8)
axes[0].yaxis.set_tick_params(labelsize=8)
axes[0].set_xticklabels(['P\'','LR','Albedo','WV','Cloud','AHT','OHT'],fontsize=8)
axes[0].set_title('(a) First 3 months',fontsize=8)
axes[0].grid(linestyle=(0,(1,5)))
axes[0].set_ylabel('Warming contribution (K)',fontsize=8)

axes[1].hlines(0,-1,7,color='black')
axes[1].errorbar(xs,mean1yr,yerr=error1yr,ecolor=colors,fmt='none')
axes[1].scatter(xs,mean1yr,c=colors)
axes[1].set_xlim([-0.5,6.5])
axes[1].set_xticks(xs)
# axes[1].set_yticks(np.arange(-1.5,1.6,0.5))
# axes[1].set_ylim([-1.5,1.5])
axes[1].xaxis.set_tick_params(labelsize=8)
axes[1].yaxis.set_tick_params(labelsize=8)
axes[1].set_xticklabels(['P\'','LR','Albedo','WV','Cloud','AHT','OHT'],fontsize=8)
axes[1].set_title('(b) First year',fontsize=8)
axes[1].grid(linestyle=(0,(1,5)))
axes[1].set_ylabel('Warming contribution (K)',fontsize=8)

axes[2].hlines(0,-1,7,color='black')
axes[2].errorbar(xs,mean30yr,yerr=error30yr,ecolor=colors,fmt='none')
axes[2].scatter(xs,mean30yr,c=colors)
axes[2].set_xlim([-0.5,6.5])
axes[2].set_xticks(xs)
# axes[2].set_yticks(np.arange(-1.5,1.6,0.5))
# axes[2].set_ylim([-1.5,1.5])
axes[2].xaxis.set_tick_params(labelsize=8)
axes[2].yaxis.set_tick_params(labelsize=8)
axes[2].set_xticklabels(['P\'','LR','Albedo','WV','Cloud','AHT','OHT'],fontsize=8)
axes[2].set_title('(c) Last 30 years',fontsize=8)
axes[2].grid(linestyle=(0,(1,5)))
axes[2].set_ylabel('Warming contribution (K)',fontsize=8)

plt.tight_layout()
plt.show()
# plt.savefig('/home/tylerj/CMIP5_AA_final/Fig_S2_060920.svg')

<Figure size 266.4x432 with 3 Axes>

In [32]:
fig,axes = plt.subplots(nrows=3,figsize=(3.7,6))
colors = ['darkorange','green','red','purple','turquoise','orchid','Navy']
feedbacks = ['planck_LWAS','lapse_LWAS','albedo_SWAS','wv_NETAS','cloud_NET','atmos_conv','ocean_conv']
for i in range(len(feedbacks)):
    if(feedbacks[i]!='ocean_conv'):
        data = diff[feedbacks[i]].isel(time=slice(0,3)).mean(dim='time')
        xs = np.full_like(data,i)
        color = colors[i]
        axes[0].scatter(xs,data,c=color,s=20,alpha=0.4)
        mean = diff[feedbacks[i]].isel(time=slice(0,3)).mean(dim=['time','model'])
        axes[0].scatter(i,mean,s=120,c=color,marker='_',linewidth=2.5)
    else:
        data = diff_ml.isel(time=slice(0,3)).mean(dim='time')
        xs = np.full_like(data,i)
        color = colors[i]
        axes[0].scatter(xs,data,c=color,s=20,alpha=0.4)
        mean = diff_ml.isel(time=slice(0,13)).mean(dim=['time','model'])
        axes[0].scatter(i,mean,s=120,c=color,marker='_',linewidth=2.5)
        
axes[0].hlines(0,-1,7,color='black')
axes[0].set_xlim([-0.5,6.5])
axes[0].set_xticks(np.arange(0,7,1))
# axes[0].set_yticks(np.arange(-1.5,1.6,0.5))
# axes[0].set_ylim([-1.5,1.5])
axes[0].xaxis.set_tick_params(labelsize=8)
axes[0].yaxis.set_tick_params(labelsize=8)
axes[0].set_xticklabels(['P\'','LR','Albedo','WV','Cloud','AHT','OHT'],fontsize=8)
axes[0].set_title('(a) First 3 months',fontsize=8)
axes[0].grid(linestyle=(0,(1,5)))
axes[0].set_ylabel('Warming contribution (K)',fontsize=8)

for i in range(len(feedbacks)):
    if(feedbacks[i]!='ocean_conv'):
        data = diff[feedbacks[i]].isel(time=slice(0,12)).mean(dim='time')
        xs = np.full_like(data,i)
        color = colors[i]
        axes[1].scatter(xs,data,c=color,s=20,alpha=0.4)
        mean = diff[feedbacks[i]].isel(time=slice(0,12)).mean(dim=['time','model'])
        axes[1].scatter(i,mean,s=120,c=color,marker='_',linewidth=2.5)
    else:
        data = diff_ml.isel(time=slice(0,12)).mean(dim='time')
        xs = np.full_like(data,i)
        color = colors[i]
        axes[1].scatter(xs,data,c=color,s=20,alpha=0.4)
        mean = diff_ml.isel(time=slice(0,12)).mean(dim=['time','model'])
        axes[1].scatter(i,mean,s=120,c=color,marker='_',linewidth=2.5)

axes[1].hlines(0,-1,7,color='black')
axes[1].set_xlim([-0.5,6.5])
axes[1].set_xticks(np.arange(0,7,1))
# axes[1].set_yticks(np.arange(-1.5,1.6,0.5))
# axes[1].set_ylim([-1.5,1.5])
axes[1].xaxis.set_tick_params(labelsize=8)
axes[1].yaxis.set_tick_params(labelsize=8)
axes[1].set_xticklabels(['P\'','LR','Albedo','WV','Cloud','AHT','OHT'],fontsize=8)
axes[1].set_title('(b) First year',fontsize=8)
axes[1].grid(linestyle=(0,(1,5)))
axes[1].set_ylabel('Warming contribution (K)',fontsize=8)


for i in range(len(feedbacks)):
    if(feedbacks[i]!='ocean_conv'):
        data = diff[feedbacks[i]].isel(time=slice(1200,1800)).mean(dim='time')
        xs = np.full_like(data,i)
        color = colors[i]
        axes[2].scatter(xs,data,c=color,s=20,alpha=0.4)
        mean = diff[feedbacks[i]].isel(time=slice(1200,1800)).mean(dim=['time','model'])
        axes[2].scatter(i,mean,s=120,c=color,marker='_',linewidth=2.5)
    else:
        data = diff_ml.isel(time=slice(1200,1800)).mean(dim='time')
        xs = np.full_like(data,i)
        color = colors[i]
        axes[2].scatter(xs,data,c=color,s=20,alpha=0.4)
        mean = diff_ml.isel(time=slice(1200,1800)).mean(dim=['time','model'])
        axes[2].scatter(i,mean,s=120,c=color,marker='_',linewidth=2.5)

axes[2].hlines(0,-1,7,color='black')
axes[2].set_xlim([-0.5,6.5])
axes[2].set_xticks(np.arange(0,7,1))
# axes[2].set_yticks(np.arange(-1.5,1.6,0.5))
# axes[2].set_ylim([-1.5,1.5])
axes[2].xaxis.set_tick_params(labelsize=8)
axes[2].yaxis.set_tick_params(labelsize=8)
axes[2].set_xticklabels(['P\'','LR','Albedo','WV','Cloud','AHT','OHT'],fontsize=8)
axes[2].set_title('(c) Last 30 years',fontsize=8)
axes[2].grid(linestyle=(0,(1,5)))
axes[2].set_ylabel('Warming contribution (K)',fontsize=8)

plt.tight_layout()
# plt.show()
plt.savefig('/home/tylerj/CMIP5_AA_final/all_models_feedbacks_062320.svg')

<Figure size 266.4x432 with 3 Axes>

In [56]:
diff.lapse_LWAS.isel(time=slice(0,3)).mean(dim='time').std(dim='model')/np.sqrt(21)

<xarray.DataArray 'lapse_LWAS' ()>
array(0.11196829)